## First method to find sum, mean, max and mean
# Using Segment Tree

In [1]:
# Reading the data from given csv the pandas
import pandas as pd
dframe = pd.read_csv('Data required for the assignmnet.csv')
dframe.head()

,timestamp,Mumb,Chennai,Khar,Delh
0,1360009758,57.536559,51.338224,40.172635,40.274136
1,1360009579,51.432071,42.668862,52.881400,46.432223
2,1360009535,43.755586,44.066454,49.656046,44.061495
3,1360009618,48.483868,56.036239,44.631650,43.831369
4,1360009677,41.952270,53.648003,41.611338,44.356289


In [2]:
# Checking for presence of missing values
dframe.isna().sum()

,0
timestamp,0
Mumb,1
Chennai,15
Khar,8
Delh,6


In [3]:
# Imputing the missing values with mean
dframe.fillna(dframe.mean(), inplace=True)

# Storing the data after imputation
dframe.to_csv('imputed_data.csv', index=False)

In [4]:
# Checking for duplicate timestamps
dframe['timestamp'].duplicated().sum()

14698

In [5]:
# Storing the name of all cities
cities = dframe.columns[1:].to_list()
print(cities)

['Mumb', 'Chennai', 'Khar', 'Delh']


In [6]:
#Checking the type of all values in the dataset
dframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  14999 non-null  int64  
 1   Mumb       14999 non-null  float64
 2   Chennai    14999 non-null  float64
 3   Khar       14999 non-null  float64
 4   Delh       14999 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 586.0 KB


In [7]:
# Variable to store the minimum and maximum timestamps
min_time_stamp = dframe['timestamp'][0]
max_time_stamp = dframe['timestamp'][0]

# Creating the time_stamps to store the combined values of duplicated timestamps
time_stamps = {}

for row in range(dframe.shape[0]):
  cur_time_stamp = dframe.loc[row, 'timestamp']
  min_time_stamp = min(min_time_stamp, cur_time_stamp)
  max_time_stamp = max(max_time_stamp, cur_time_stamp)
  if cur_time_stamp not in time_stamps:
    time_stamps[cur_time_stamp] = {'count': 1}  # Count is used to calculate the mean
    for city in cities:
      time_stamps[cur_time_stamp][city] = {}
      time_stamps[cur_time_stamp][city]['min'] = dframe.loc[row, city]     # Min stores the min of all duplicated timestamps
      time_stamps[cur_time_stamp][city]['max'] = dframe.loc[row, city]     # Max stores the max of all duplicated timestamps
      time_stamps[cur_time_stamp][city]['sum'] = dframe.loc[row, city]     # Sum stores the sum of all duplicated timestamps
  else:
    time_stamps[cur_time_stamp]['count'] += 1
    for city in cities:
      time_stamps[cur_time_stamp][city]['min'] = min(time_stamps[cur_time_stamp][city]['min'], dframe.loc[row, city])
      time_stamps[cur_time_stamp][city]['max'] = max(time_stamps[cur_time_stamp][city]['max'], dframe.loc[row, city])
      time_stamps[cur_time_stamp][city]['sum'] += dframe.loc[row, city]


In [8]:
# Segement tree implementation with array
class Segment_Tree:
  def __init__(self, first_timestamp, last_timestamp, time_stamps, length, cities):
    self.time_stamps = time_stamps
    self.first_timestamp = first_timestamp
    self.last_timestamp = last_timestamp
    self.cities = cities
    self.tree_arr = [{}] * (4 * length)
    self.build_tree(0, first_timestamp, last_timestamp)

  # Function to combine the node stats from left and right sub tree
  def combine_node(self, left_node, right_node):
    if left_node == {}:
      return right_node
    elif right_node == {}:
      return left_node

    cur_node = {}
    for city in self.cities:
        cur_node[city] = {}
        cur_node[city]['min'] = min(left_node[city]['min'], right_node[city]['min'])
        cur_node[city]['max'] = max(left_node[city]['max'], right_node[city]['max'])
        cur_node[city]['sum'] = left_node[city]['sum'] + right_node[city]['sum']
    cur_node['count'] = left_node['count'] + right_node['count']
    return cur_node

  #Function to build the segment tree
  def build_tree(self, ind, start_timestamp, end_timestamp):
    if start_timestamp == end_timestamp:                        # If the leaf node is reached, the value of specific time stamp is stored.
      self.tree_arr[ind] = self.time_stamps[start_timestamp]
      return self.tree_arr[ind]

    mid = (start_timestamp + end_timestamp) // 2
    left = 2 * ind + 1
    right = 2 * ind + 2
    left_node = self.build_tree(left, start_timestamp, mid)     # Building the left subtree
    right_node = self.build_tree(right, mid+1, end_timestamp)   # Building the right subtree
    self.tree_arr[ind] = self.combine_node(left_node, right_node)   # Combining the left and right subtree nodes while backtracking, to get stats of range of nodes
    return self.tree_arr[ind]

  #Function to query the segment tree
  def query(self, start_timestamp, end_timestamp, ind, node_left , node_right):
    if start_timestamp > node_right or end_timestamp < node_left:            # If timestamps dont lie within stored intervals
      return {}
    elif start_timestamp <= node_left and end_timestamp >= node_right:       # If the current node contain the entire timestamp
      return self.tree_arr[ind]
    else:                                                                    # If the above both cond fails, get stats from left and right subtree
      mid = (node_left + node_right) // 2
      left = 2 * ind + 1
      right = 2 * ind + 2
      left_node = self.query(start_timestamp, end_timestamp, left, node_left, mid)
      right_node = self.query(start_timestamp, end_timestamp, right, mid + 1, node_right)
      return self.combine_node(left_node, right_node)

  # Function to get required stat: sum, mean, min, max from segment tree
  def get_stats(self, start_timestamp, end_timestamp, city):
    node = self.query(start_timestamp, end_timestamp, 0, self.first_timestamp, self.last_timestamp)

    if node == {}:
      print("No data found")
      return

    sum = node[city]['sum']
    mean = sum / node['count']
    min = node[city]['min']
    max = node[city]['max']
    print(f"The sum in interval {start_timestamp} and {end_timestamp} of {city} is {sum:.2f}")
    print(f"The mean in interval {start_timestamp} and {end_timestamp} of {city} is {mean:.2f}")
    print(f"The min in interval {start_timestamp} and {end_timestamp} of {city} is {min:.4f}")
    print(f"The max in interval {start_timestamp} and {end_timestamp} of {city} is {max:.4f}")



In [9]:
length = max_time_stamp - min_time_stamp + 1
seg_tree_instance = Segment_Tree(min_time_stamp, max_time_stamp, time_stamps, length, cities)

In [10]:
seg_tree_instance.get_stats(1360009471, 1360009471, 'Mumb')

The sum in interval 1360009471 and 1360009471 of Mumb is 1193.44
The mean in interval 1360009471 and 1360009471 of Mumb is 49.73
The min in interval 1360009471 and 1360009471 of Mumb is 42.2733
The max in interval 1360009471 and 1360009471 of Mumb is 60.3637


In [11]:
seg_tree_instance.get_stats(1360009475, 1360009477, 'Mumb')

The sum in interval 1360009475 and 1360009477 of Mumb is 7590.71
The mean in interval 1360009475 and 1360009477 of Mumb is 50.60
The min in interval 1360009475 and 1360009477 of Mumb is 40.5473
The max in interval 1360009475 and 1360009477 of Mumb is 60.4673


In [12]:
seg_tree_instance.get_stats(1360009481, 1360009491, 'Khar')

The sum in interval 1360009481 and 1360009491 of Khar is 27783.40
The mean in interval 1360009481 and 1360009491 of Khar is 50.52
The min in interval 1360009481 and 1360009491 of Khar is 40.5739
The max in interval 1360009481 and 1360009491 of Khar is 60.4972


In [13]:
seg_tree_instance.get_stats(1360009471, 1360009771, 'Delh')

The sum in interval 1360009471 and 1360009771 of Delh is 753593.06
The mean in interval 1360009471 and 1360009771 of Delh is 50.24
The min in interval 1360009471 and 1360009771 of Delh is 40.1191
The max in interval 1360009471 and 1360009771 of Delh is 60.6020


## First Method above can query all sum, mean, max and min in O(log n) using segment tree
## ------------------------------------------------------ End of Assignment ---------------------------------------------------------------


## Second Method below queries sum and mean in time complexity of O(1) using prefix sum

In [14]:
dframe['timestamp'].unique().shape

(301,)

In [15]:
#Sort the rows according to time stamp and reset index
dframe2 = dframe.sort_values(by=['timestamp'])
dframe2 = dframe2.reset_index(drop=True)
dframe2.head()


,timestamp,Mumb,Chennai,Khar,Delh
0,1360009471,44.212887,43.727827,41.208346,55.174262
1,1360009471,46.662031,57.391402,40.694280,50.951203
2,1360009471,54.259282,45.678202,55.419766,46.414326
3,1360009471,53.271850,41.807774,50.255977,42.110217
4,1360009471,51.889060,43.480799,56.232718,56.036354


In [16]:
city_names = dframe2.columns[1:].to_list()
print(city_names)

['Mumb', 'Chennai', 'Khar', 'Delh']


In [17]:
# Storing the first and last index for each timestamp and also calculating the prefix sum
time_stamps_index = {}
for row in range(dframe2.shape[0]):
  time_stamp = dframe2.loc[row, 'timestamp']

  if time_stamp not in time_stamps_index:
    time_stamps_index[time_stamp] = {}
    time_stamps_index[time_stamp]['first_index'] = row

  time_stamps_index[time_stamp]['last_index'] = row

  for city in city_names:
    if row > 0:
      dframe2.loc[row, city] += dframe2.loc[row-1, city]

In [18]:
dframe2.head()

,timestamp,Mumb,Chennai,Khar,Delh
0,1360009471,44.212887,43.727827,41.208346,55.174262
1,1360009471,90.874918,101.119230,81.902626,106.125465
2,1360009471,145.134201,146.797432,137.322392,152.539791
3,1360009471,198.406050,188.605206,187.578370,194.650009
4,1360009471,250.295110,232.086005,243.811087,250.686362


In [19]:
dframe2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  14999 non-null  int64  
 1   Mumb       14999 non-null  float64
 2   Chennai    14999 non-null  float64
 3   Khar       14999 non-null  float64
 4   Delh       14999 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 586.0 KB


In [20]:
dframe2.shape

(14999, 5)

In [21]:
def get_stats(city, time_stamp_start, time_stamp_end):
  # Edge case if time stamp query out of stored timestamp intervals
  if time_stamp_start > dframe2.iloc[-1, 0] or time_stamp_end < dframe2.iloc[0, 0]:
      print("No data found")
      return

  # Trimming the time stamps if it crosses boundaries
  if time_stamp_start < dframe2.iloc[0, 0]:
    time_stamp_start = dframe2.iloc[0, 0]
  if time_stamp_end > dframe2.iloc[-1, 0]:
    time_stamp_end = dframe2.iloc[-1, 0]

  first_index = time_stamps_index[time_stamp_start]['first_index']
  last_index = time_stamps_index[time_stamp_end]['last_index']
  sum = 0

  # Edge case 2 if the the first index is zero
  if first_index == 0:
    sum = dframe2.loc[last_index, city]
  else:
    sum = dframe2.loc[last_index, city] - dframe2.loc[first_index-1, city]

  mean = sum / (last_index - first_index + 1)

  print(f"The sum in interval {time_stamp_start} and {time_stamp_end} of {city} is {sum:.2f}")
  print(f"The mean in interval {time_stamp_start} and {time_stamp_end} of {city} is {mean:.2f}")


In [22]:
get_stats('Mumb', 1360009471, 1360009471)

The sum in interval 1360009471 and 1360009471 of Mumb is 1193.44
The mean in interval 1360009471 and 1360009471 of Mumb is 49.73


In [23]:
get_stats('Mumb', 1360009475, 1360009477)

The sum in interval 1360009475 and 1360009477 of Mumb is 7590.71
The mean in interval 1360009475 and 1360009477 of Mumb is 50.60


In [24]:
get_stats('Khar', 1360009481, 1360009491)

The sum in interval 1360009481 and 1360009491 of Khar is 27783.40
The mean in interval 1360009481 and 1360009491 of Khar is 50.52


In [25]:
get_stats('Delh', 1360009471, 1360009771)

The sum in interval 1360009471 and 1360009771 of Delh is 753593.06
The mean in interval 1360009471 and 1360009771 of Delh is 50.24
